# Project 4

Implementating, evaluating and comparing on  algorithms: Fairness Beyond Disparate Treatment & Disparate Impact: Learning Classification without Disparate Mistreatment (DM and DM-sen) and Handling Conditional Discrimination (LM and LPS)

Team members: 

- Sarah Kurihara

- Varchasvi Vedula

- Wenhui Fang

- Krista Zhang

- Sharon Meng


# 1. Setup

### Import packages

In [2]:
import warnings

import time,sys
import zipfile
import random
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from tensorflow import keras
from keras.layers import Dense, Input
from tensorflow.keras import Model
import scipy.stats as ss
import dccp
import cvxpy as cvx
from cvxpy import *

warnings.filterwarnings('ignore')

### Load Dataset and Data Preprocessing



In [65]:
start=time.time()
data = pd.read_csv('../data/compas-scores-two-years.csv')
#print(data.shape)
# filter out groups other than African-American and Caucasian and set them as 0-1
data = data[(data['race']=='African-American') | (data['race']=='Caucasian')]
data['race'].loc[data['race']=='Caucasian'] = 1
data['race'].loc[data['race']=='African-American'] = 0
#print(data.shape)

nan = (data.isnull().sum()/len(data))
nan = nan[nan > 0.15].sort_values()
nan_var = list(nan.index)
data = data.drop(columns=nan_var)

data['c_jail_in'] = pd.to_datetime(data['c_jail_in'])
data['c_jail_out'] = pd.to_datetime(data['c_jail_out'])
data['los'] = np.log((data['c_jail_out']-data['c_jail_in']).astype('timedelta64[h]')+1)#use log hours
data['in_custody'] = pd.to_datetime(data['in_custody'])
data['out_custody'] = pd.to_datetime(data['out_custody'])
data['custody'] = np.log((data['out_custody']-data['in_custody']).astype('timedelta64[h]')+1)
data['lasts'] = np.log(data['end']-data['start']+1)
data['c_days_from_compas'] = np.log(data['c_days_from_compas']+1)

#filter out useless variables including high correlation and string type
useless_var = ['id','name','first','last','compas_screening_date','dob','age_cat','days_b_screening_arrest',
               'c_jail_in','c_jail_out','c_case_number','c_charge_desc','is_recid',
               'type_of_assessment','screening_date','v_type_of_assessment',
               'v_screening_date','in_custody','out_custody','score_text','v_score_text',
               'decile_score.1','v_decile_score','priors_count.1','start','end']
data = data.drop(columns=useless_var)
data = data[data['los']!=float('-inf')]
data = data[data['custody']!=float('-inf')]
data = data[data['lasts']!=float('-inf')]

#one hot encoding on several features:sex,age_cat,c_charge_degree
data['sex'].loc[data['sex']=='Male']= 1
data['sex'].loc[data['sex']=='Female']= 0
data['c_charge_degree'].loc[data['c_charge_degree']=='M']= 1
data['c_charge_degree'].loc[data['c_charge_degree']=='F']= 0
#data.to_csv('./data/compas_preproc.csv',index=False,header=True)
del nan_var, useless_var

#data = data[['age','race','sex','decile_score','priors_count','los','c_charge_degree','two_year_recid']]
data = data.dropna()#6150*23->5730*16
#print(data.shape)
print(data.head(5))

X = data.drop(columns='two_year_recid')
features = list(X.columns)

X.index = range(data.shape[0])
#As age, priors_count, los are continuous variables, we can scale them
X_cont = X[['age', 'juv_fel_count', 'decile_score', 'juv_misd_count', 'juv_other_count', 'priors_count', 'c_days_from_compas', 'los', 'custody', 'lasts']]
X_cate = X[['sex', 'race', 'c_charge_degree', 'is_violent_recid', 'event']]
X_cont = pd.DataFrame(StandardScaler().fit_transform(X_cont),columns=['age', 'juv_fel_count', 'decile_score', 'juv_misd_count', 'juv_other_count', 'priors_count', 'c_days_from_compas', 'los', 'custody', 'lasts'])
#X_cont = X[['age', 'decile_score', 'priors_count', 'los']]
#X_cate = X[['sex', 'race', 'c_charge_degree']]
#X_cont = pd.DataFrame(StandardScaler().fit_transform(X_cont),columns=['age', 'decile_score', 'priors_count', 'los'])

X_df = pd.concat([X_cate,X_cont],axis=1)
#X['decile_score'] = X['decile_score']/10
y_df = data["two_year_recid"]
# convert class label 0 to -1 so as to add sign in distance
#y[y==0] = -1
features = list(X.columns)

X = np.asarray(X_df).astype('float32')
y = np.asarray(y_df).astype('float32')

del X_cate,X_cont

  sex  age race  juv_fel_count  decile_score  juv_misd_count  juv_other_count  \
1   1   34    0              0             3               0                0   
2   1   24    0              0             4               0                1   
6   1   41    1              0             6               0                0   
8   0   39    1              0             1               0                0   
9   1   21    1              0             3               0                0   

   priors_count  c_days_from_compas c_charge_degree  is_violent_recid  event  \
1             0            0.693147               0                 1      1   
2             4            0.693147               0                 0      0   
6            14            0.693147               0                 0      1   
8             0            0.693147               1                 0      0   
9             1            5.733341               0                 1      1   

   two_year_recid       los   cu

In [66]:
#Use 5:1:1 as the ratio of train:val:test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=800, random_state=3)
X_train, X_val, y_train, y_val = train_test_split(X_train,y_train,test_size=800,random_state=3)

# For A6
X_train_df, X_test_df, y_train_df, y_test_df = train_test_split(X_df, y_df, test_size=800, random_state=3)
X_train_df, X_val_df, y_train_df, y_val_df = train_test_split(X_train_df,y_train_df,test_size=800,random_state=3)

train_size = len(y_train)
train_idx_AA = np.array(range(train_size))[X_train[:,1]==0.0]
train_idx_C = np.array(range(train_size))[X_train[:,1]==1.0]
test_size = len(y_test)
test_idx_AA = np.array(range(test_size))[X_test[:,1]==0.0]
test_idx_C = np.array(range(test_size))[X_test[:,1]==1.0]

# For A6
X_train_df.race = abs(X_train_df.race-1)
X_test_df.race = abs(X_test_df.race-1)
X_val_df.race = abs(X_val_df.race-1)
X_train_df.race = abs(X_train_df.c_charge_degree-1)
X_test_df.race = abs(X_test_df.c_charge_degree-1)
X_val_df.race = abs(X_val_df.c_charge_degree-1)
train = pd.concat([X_train_df.reset_index(drop=True), y_train_df.reset_index(drop=True)], axis = 1)
test = pd.concat([X_test_df.reset_index(drop=True), y_test_df.reset_index(drop=True)], axis = 1)

print('\n',"#"*80,'\n',' '*20," Split up Train-Validation-Test sets ",'\n',"#"*80,'\n')
print(" X_train size: ", X_train.shape, ",      y_train size: ", y_train.shape, '\n',
      "X_validation size: ", X_val.shape, ",  y_validation size: ", y_val.shape, '\n',
      "X_test size: ", X_test.shape, ',        y_test size: ',y_test.shape)
print(" X_train_AA size: ", X_train[train_idx_AA].shape, ",   X_train_C size: ", X_train[train_idx_C].shape, '\n',
      "X_test_AA size: ", X_test[test_idx_AA].shape, ',     X_test_C size: ',X_test[test_idx_C].shape, '\n',
      "Ratio:", X_train[train_idx_AA].shape[0]/X_train[train_idx_C].shape[0],X_test[test_idx_AA].shape[0]/X_test[test_idx_C].shape[0])
print('\n',"#"*80)


 ################################################################################ 
                       Split up Train-Validation-Test sets  
 ################################################################################ 

 X_train size:  (4130, 15) ,      y_train size:  (4130,) 
 X_validation size:  (800, 15) ,  y_validation size:  (800,) 
 X_test size:  (800, 15) ,        y_test size:  (800,)
 X_train_AA size:  (2472, 15) ,   X_train_C size:  (1658, 15) 
 X_test_AA size:  (478, 15) ,     X_test_C size:  (322, 15) 
 Ratio: 1.490952955367913 1.484472049689441

 ################################################################################


# 2. A4

## 2.1 Baseline Model And Evaluation

In [40]:
#If use Neural Network as base model
def base_nn_model(X_in,y_in,X_val,y_val):
    feature = Input(X_in.shape[1],)
    y = Dense(2,"softmax")(feature)
    model = Model(feature,y)
    
    adam = tf.keras.optimizers.Adam(0.001)
    loss = keras.losses.BinaryCrossentropy(from_logits=True)
    metric = [tf.keras.metrics.BinaryAccuracy()]
    #,tf.keras.metrics.FalsePositives()，tf.keras.metrics.FalseNegatives()
    model.compile(optimizer=adam, loss=loss, metrics=metric)
    model.fit(X_in,tf.one_hot(y_in,2),epochs=10,batch_size=10,validation_data=(X_val,tf.one_hot(y_val,2)))
    return model

def evaluation(model,X,y):
    y_pred = model.predict(X)
    y_pred = np.argmax(np.round(y_pred), axis=1)
    y_pred_AA, y_test_AA = y_pred[test_idx_AA], y[test_idx_AA]
    y_pred_C, y_test_C = y_pred[test_idx_C], y[test_idx_C]
    acc = model.evaluate(X, tf.one_hot(y,2))[1]
    FPR_all = sum(y_pred[y==0]==1)/len(y[y==0])
    FNR_all = sum(y_pred[y==1]==0)/len(y[y==1])
    FPR_AA = sum(y_pred_AA[y_test_AA==0]==1)/len(y_test_AA[y_test_AA==0])
    FNR_AA = sum(y_pred_AA[y_test_AA==1]==0)/len(y_test_AA[y_test_AA==1])
    FPR_C = sum(y_pred_C[y_test_C==0]==1)/len(y_test_C[y_test_C==0])
    FNR_C = sum(y_pred_C[y_test_C==1]==0)/len(y_test_C[y_test_C==1])
    pred_p_AA, pred_p_C = np.mean(y_pred_AA==1), np.mean(y_pred_C==1)
    acc_AA, acc_C = np.mean(y_pred_AA == y_test_AA), np.mean(y_pred_C == y_test_C)
    print('\n',"#"*80)
    print('The accuracy of baseline model NN is: %3f.'%(acc))
    print('The False Positive Rate for overall population is: %3f.'%FPR_all)
    print('The False Negative Rate for overall population is: %3f.'%FNR_all)
    print("Specifically:")
    print('Parity Check: The rate of positive estimate for African American and Caucasian are %3f and %3f, and D_par=%3f.'%(pred_p_AA,pred_p_C,pred_p_AA-pred_p_C))
    print('Calibration Check: The rate of correct estimate for African American and Caucasian are %3f and %3f, and D_cal=%3f.'%(acc_AA,acc_C,acc_AA-acc_C))
    print('The False Positive Rate for African American and Caucasian are %3f and %3f, and D_FPR=%3f.'%(FPR_AA,FPR_C,FPR_AA-FPR_C))
    print('The False Negative Rate for African American and Caucasian are %3f and %3f, and D_FNR=%3f.'%(FNR_AA,FNR_C,FNR_AA-FNR_C))
    print('\n',"#"*80)

In [41]:
#If we don't drop 'race' in the X_train and X_test:
NN1 = base_nn_model(X_train,y_train,X_val,y_val)
evaluation(NN1,X_test,y_test)
#If we drop 'race' in the X_train and X_test:
NN2 = base_nn_model(np.delete(X_train,0,1),y_train,np.delete(X_val,0,1),y_val)
evaluation(NN2,np.delete(X_test,1,1),y_test)

Epoch 1/10


2022-04-10 15:34:11.319385: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


413/413 [==============================] - 0s 659us/step - loss: 0.7449 - binary_accuracy: 0.6378 - val_loss: 0.5804 - val_binary_accuracy: 0.8150
Epoch 2/10
413/413 [==============================] - 0s 439us/step - loss: 0.5076 - binary_accuracy: 0.8441 - val_loss: 0.4428 - val_binary_accuracy: 0.8788
Epoch 3/10
413/413 [==============================] - 0s 435us/step - loss: 0.4095 - binary_accuracy: 0.8726 - val_loss: 0.3736 - val_binary_accuracy: 0.8875
Epoch 4/10
413/413 [==============================] - 0s 437us/step - loss: 0.3532 - binary_accuracy: 0.8818 - val_loss: 0.3278 - val_binary_accuracy: 0.8963
Epoch 5/10
413/413 [==============================] - 0s 435us/step - loss: 0.3145 - binary_accuracy: 0.8998 - val_loss: 0.2950 - val_binary_accuracy: 0.9062
Epoch 6/10
413/413 [==============================] - 0s 436us/step - loss: 0.2864 - binary_accuracy: 0.9068 - val_loss: 0.2709 - val_binary_accuracy: 0.9112
Epoch 7/10
413/413 [==============================] - 0s 439us/

Then we custom constraint and loss to the base model:

In [42]:
def DFR(model,X,y,type):
    '''
    type: str in ['dfnr','dfpr','both']
    '''
    if type!='dfnr' and type!='dfpr' and type!='both':
        return None
    size = len(y)
    idx_AA = np.array(range(size))[X[:,1]==0.0]
    idx_C = np.array(range(size))[X[:,1]==1.0]
    y_pred = model.predict(X)
    y_pred = np.argmax(np.round(y_pred), axis=1)
    y_pred_AA, y_AA = y_pred[idx_AA], y[idx_AA]
    y_pred_C, y_C = y_pred[idx_C], y[idx_C] 
    FPR_AA = sum(y_pred_AA[y_AA==0]==1)/len(y_AA[y_AA==0])
    FNR_AA = sum(y_pred_AA[y_AA==1]==0)/len(y_AA[y_AA==1])
    FPR_C = sum(y_pred_C[y_C==0]==1)/len(y_C[y_C==0])
    FNR_C = sum(y_pred_C[y_C==1]==0)/len(y_C[y_C==1])
    dfnr = FNR_AA-FNR_C
    dfpr = FPR_AA-FPR_C
    if type=='dfnr':
        return dfnr
    elif type=='dfpr':
        return dfpr
    else:
        return dfnr,dfpr
dfnr,dfpr = DFR(NN1,X_test,y_test,'both')
print(dfnr,dfpr)

-0.060218639985176944 0.04852710314895188


In [43]:
def new_training_groups(model, X_train, y_train):
    '''
    X       n*d
    model.predict(X)    n*2
    y       n,
    delta   c,
    dfr   "dfpr","dfnr","both"
    '''
    
    #split training sets according to sensitive variable
    X_train_AA = X_train[np.array(X_train[:,1] == 0.0)]
    y_train_AA = y_train[np.array(X_train[:,1] == 0.0)]
    X_train_C = X_train[np.array(X_train[:,1] == 1.0)]
    y_train_C = y_train[np.array(X_train[:,1] == 1.0)]
    #get the ones with wrong prediction in discriminated group
    dn,dp = DFR(model,X_train,y_train,type="both")
    if dp>0: d = 0
    else: d = 1

    if d == 0:
        #take penalized trainers
        y_pred_AA = np.argmax(model.predict(X_train_AA),axis = 1)
        y_diff_AA = y_train_AA-y_pred_AA
        X_train_penalized = X_train_AA[y_diff_AA != 0.0]
        y_train_penalized = y_train_AA[y_diff_AA != 0.0]
        # safe trainers
        X_train_clean = X_train_AA[y_diff_AA == 0.0]
        y_train_clean = y_train_AA[y_diff_AA == 0.0]
        #make new
        X_train_clean = np.concatenate((X_train_clean,X_train_C),axis=0)
        y_train_clean = np.concatenate((y_train_clean,y_train_C),axis=0)
    
    else:
        #reverse the steps above for train set 1
        y_pred_C = np.argmax(model.predict(X_train_C),axis = 1)
        y_diff_C = y_train_C-y_pred_C
        X_train_penalized = X_train_C[y_diff_C != 0.0]
        y_train_penalized = y_train_C[y_diff_C != 0.0]
        # safe trainers
        X_train_clean = X_train_C[y_diff_C == 0.0]
        y_train_clean = y_train_C[y_diff_C == 0.0]
        #make new
        X_train_clean = np.concatenate((X_train_clean,X_train_AA),axis=0)
        y_train_clean = np.concatenate((y_train_clean,y_train_AA),axis=0)
    
    #X_train_penalized = tf.convert_to_tensor(X_train_penalized, dtype=tf.float32)
    #X_train_safe = tf.convert_to_tensor(X_train_safe, dtype=tf.float32)
    return X_train_clean , y_train_clean , X_train_penalized , y_train_penalized, dn, dp


Here we use Neural network as the base model, the result is unlikely to be reimplemented, but similar result will come out.

**Note: When few features are used in the model, the model and its prediction are unstable.

In [44]:
#########################[DON'T RUN THIS CELL IF UNNECESSARY]#########################

## initialization
np.random.seed(7777) 
model =  base_nn_model(X_train, y_train, X_val, y_val)

#initialized C and delta
C = 1
delta = 0.2
iter = 20

# new training groups
X_ts, y_ts, X_tp, y_tp, dn, dp = new_training_groups(model, X_train, y_train)
feature = Input(X_train.shape[1],)
y = Dense(2,"softmax")(feature)
mod_loop = Model(feature,y)
adam = tf.keras.optimizers.Adam(0.001)
loss = keras.losses.BinaryCrossentropy(from_logits=True)
metric = [tf.keras.metrics.BinaryAccuracy()]

def penal_loss(y_true,y_pred):
  return loss(tf.one_hot(y_tp,2), mod_loop(X_tp))

def clean_loss(y_true,y_pred):  
  return loss(tf.one_hot(y_ts,2), mod_loop(X_ts))

count = 0
# start while loop
while (count==0 or count%2==1 or abs(dp)>0.05) and count<iter: 
    # or examine dn, here count%2==1 used to control accuracy in case the accuracy is below 0.5 and one of the overall fpr/fnr will be close to 1
    C = C+delta
    #print('Count:%d'%count)
    mod_loop.compile(optimizer=adam,loss=[penal_loss, clean_loss],loss_weights=[C,1],metrics=metric)
    mod_loop.fit(X_train, tf.one_hot(y_train,2), epochs=10, validation_data=(X_val,tf.one_hot(y_val,2)))
    X_ts, y_ts, X_tp, y_tp, dn, dp = new_training_groups(mod_loop, X_train, y_train)
    #dp = DFR(model_in_loop,X_test,y_test,'dfpr')
    count+=1

Epoch 1/10
413/413 [==============================] - 0s 612us/step - loss: 0.7364 - binary_accuracy: 0.6027 - val_loss: 0.5753 - val_binary_accuracy: 0.7775
Epoch 2/10
413/413 [==============================] - 0s 436us/step - loss: 0.5112 - binary_accuracy: 0.8172 - val_loss: 0.4368 - val_binary_accuracy: 0.8662
Epoch 3/10
413/413 [==============================] - 0s 433us/step - loss: 0.4083 - binary_accuracy: 0.8663 - val_loss: 0.3637 - val_binary_accuracy: 0.8950
Epoch 4/10
413/413 [==============================] - 0s 433us/step - loss: 0.3493 - binary_accuracy: 0.8886 - val_loss: 0.3180 - val_binary_accuracy: 0.9025
Epoch 5/10
413/413 [==============================] - 0s 460us/step - loss: 0.3102 - binary_accuracy: 0.8988 - val_loss: 0.2866 - val_binary_accuracy: 0.9112
Epoch 6/10
413/413 [==============================] - 0s 470us/step - loss: 0.2822 - binary_accuracy: 0.9099 - val_loss: 0.2640 - val_binary_accuracy: 0.9125
Epoch 7/10
413/413 [==============================] 

In [45]:
#Evaluation
print(DFR(mod_loop,X_test,y_test,type='both'))
evaluation(mod_loop,X_test,y_test)

(-0.09224260391575567, 0.07008957429125497)
25/25 [==============================] - 0s 393us/step - loss: 0.2042 - binary_accuracy: 0.8363

 ################################################################################
The accuracy of baseline model NN is: 0.836250.
The False Positive Rate for overall population is: 0.057554.
The False Negative Rate for overall population is: 0.279373.
Specifically:
Parity Check: The rate of positive estimate for African American and Caucasian are 0.445607 and 0.270186, and D_par=0.175420.
Calibration Check: The rate of correct estimate for African American and Caucasian are 0.824268 and 0.854037, and D_cal=-0.029769.
The False Positive Rate for African American and Caucasian are 0.090498 and 0.020408, and D_FPR=0.070090.
The False Negative Rate for African American and Caucasian are 0.249027 and 0.341270, and D_FNR=-0.092243.

 ################################################################################


In [46]:
####################################################################################################################################
####################### IF NEED TO MODIFY CODES, USE THIS CHUNK TO RUN ABOVE OR RUN BELOW ##########################################
####################################################################################################################################

## Fairness Beyond Disparate Treatment & Disparate Impact: Learning Classification without Disparate Mistreatment ($DM_{sen}$ & $DM$)

We first implement $DM_{sen}$ as we won't do anything to the dataset:

In [67]:
#Use 4:1 as the ratio of train:test
y = data.two_year_recid
y = np.asarray(y).astype('float32')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
train_size = len(y_train)
train_idx_AA = np.array(range(train_size))[X_train[:,1]==0.0]
train_idx_C = np.array(range(train_size))[X_train[:,1]==1.0]
test_size = len(y_test)
test_idx_AA = np.array(range(test_size))[X_test[:,1]==0.0]
test_idx_C = np.array(range(test_size))[X_test[:,1]==1.0]


# For A6
X_train_df, X_test_df, y_train_df, y_test_df = train_test_split(X_df, y_df, test_size=0.2, random_state=0)

# For A6
X_train_df.race = abs(X_train_df.race-1)
X_test_df.race = abs(X_test_df.race-1)
X_val_df.race = abs(X_val_df.race-1)
X_train_df.c_charge_degree = abs(X_train_df.c_charge_degree-1)
X_test_df.c_charge_degree = abs(X_test_df.c_charge_degree-1)
X_val_df.c_charge_degree = abs(X_val_df.c_charge_degree-1)
train = pd.concat([X_train_df.reset_index(drop=True), y_train_df.reset_index(drop=True)], axis = 1)
test = pd.concat([X_test_df.reset_index(drop=True), y_test_df.reset_index(drop=True)], axis = 1)

X_train1 = np.hstack((np.ones(X_train.shape[0]).reshape(X_train.shape[0],1), X_train))
X_train1_AA = X_train1[train_idx_AA]
X_train1_C = X_train1[train_idx_C]
y_train_AA = y_train[train_idx_AA]
y_train_C = y_train[train_idx_C]
X_test1 = np.hstack((np.ones(X_test.shape[0]).reshape(X_test.shape[0],1), X_test))
X_test1_AA = X_test1[test_idx_AA]
X_test1_C = X_test1[test_idx_C]
y_test_AA = y_test[test_idx_AA]
y_test_C = y_test[test_idx_C]

print('\n',"#"*80,'\n',' '*20," Split up Train-Test sets ",'\n',"#"*80,'\n')
print(" X_train size: ", X_train1.shape, ",      y_train size: ", y_train.shape, '\n',
      "X_test size: ", X_test1.shape, ',       y_test size: ',y_test.shape)
print(" X_train_AA size: ", X_train1[train_idx_AA].shape, ",   X_train_C size: ", X_train1[train_idx_C].shape, '\n',
      "X_test_AA size: ", X_test1[test_idx_AA].shape, ',     X_test_C size: ',X_test1[test_idx_C].shape, '\n',
      "Ratio:", X_train1[train_idx_AA].shape[0]/X_train1[train_idx_C].shape[0],X_test1[test_idx_AA].shape[0]/X_test1[test_idx_C].shape[0])
print('\n',"#"*80)


 ################################################################################ 
                       Split up Train-Test sets  
 ################################################################################ 

 X_train size:  (4584, 16) ,      y_train size:  (4584,) 
 X_test size:  (1146, 16) ,       y_test size:  (1146,)
 X_train_AA size:  (2745, 16) ,   X_train_C size:  (1839, 16) 
 X_test_AA size:  (688, 16) ,     X_test_C size:  (458, 16) 
 Ratio: 1.4926590538336053 1.502183406113537

 ################################################################################


### Custom Loss: Add Constraints

From the paper, loss function is modified in logistic regression:

In [50]:
import dccp
import cvxpy as cvx
from cvxpy import *

ModuleNotFoundError: No module named 'dccp'

In [51]:
def lossfunc(X,theta,y_true):
    # This function returns the log loss.
    y_true= 2*y_true - 1 #{0,1}->{-1,1}
    log_loss = sum(logistic(multiply(-y_true, X*theta)))
    return log_loss


np.random.seed(5243)
theta = cvx.Variable(X_train1.shape[1])
theta.value = np.random.rand(theta.shape[0])

tau, mu, EPS = 0.005, 1.5, 1 
Prob1 = cvx.Problem(Minimize(lossfunc(X_train1,theta,y_train)),[]) # No constraints
             
      
print(dccp.is_dccp(Prob1))
print(theta.value)
#[0.5591043  0.9994264  0.57031546 0.43833912 0.08453454 0.05043884
# 0.91119515 0.16423428 0.3034639  0.41950956 0.85237613 0.4244003
# 0.96147514 0.26277008 0.02849745 0.61075812]
result = Prob1.solve(method='dccp', tau=tau, mu=mu, tau_max=1e10, verbose=True) #Here changes the theta.value, result avoids the output
print(theta.value)
#[-2.22198860e+00  1.75116119e-01  1.31373607e-01 -2.60536345e-01 1.53575511e+00  4.91247336e+00 
# -2.60491152e-01  4.36237090e-03  4.13966295e-01  7.39736815e-02  1.01397507e-01  2.79119037e-01
# -6.37340155e-02  3.96745359e-02  6.88413345e-02 -2.27307636e+00]

NameError: name 'cvx' is not defined

In [ ]:
def predict(X,theta):
    #y:{-1,1}->{0,1}
    d = np.dot(X,theta)
    y_pred = (np.sign(d) + 1)/2
    return y_pred

theta_star = theta.value
y_pred = predict(X_test1, theta_star)

In [ ]:
def evaluation_DM(X,y,y_pred):
    size = X.shape[0]
    idx_AA = np.array(range(size))[X[:,1]==0.0]
    idx_C = np.array(range(size))[X[:,1]==1.0]
    y_pred_AA, y_test_AA = y_pred[idx_AA], y[idx_AA]
    y_pred_C, y_test_C = y_pred[idx_C], y[idx_C]
    FPR_all = np.sum(y_pred[y==0]==1)/len(y[y==0])
    FNR_all = np.sum(y_pred[y==1]==0)/len(y[y==1])
    FPR_AA = np.sum(y_pred_AA[y_test_AA==0]==1)/len(y_test_AA[y_test_AA==0])
    FNR_AA = np.sum(y_pred_AA[y_test_AA==1]==0)/len(y_test_AA[y_test_AA==1])
    FPR_C = np.sum(y_pred_C[y_test_C==0]==1)/len(y_test_C[y_test_C==0])
    FNR_C = np.sum(y_pred_C[y_test_C==1]==0)/len(y_test_C[y_test_C==1])
    pred_p_AA, pred_p_C = np.mean(y_pred_AA==1), np.mean(y_pred_C==1)
    acc = np.sum(y_pred == y)/len(y)
    acc_AA, acc_C = np.mean(y_pred_AA == y_test_AA), np.mean(y_pred_C == y_test_C)
    print('\n',"#"*80)
    print('The accuracy of baseline model LR is: %3f.'%(acc))
    print('The False Positive Rate for overall population is: %3f.'%FPR_all)
    print('The False Negative Rate for overall population is: %3f.'%FNR_all)
    print("Specifically:")
    print('Parity Check: The rate of positive estimate for African American and Caucasian are %3f and %3f, and D_par=%3f.'%(pred_p_AA,pred_p_C,pred_p_AA-pred_p_C))
    print('Calibration Check: The rate of correct estimate for African American and Caucasian are %3f and %3f, and D_cal=%3f.'%(acc_AA,acc_C,acc_AA-acc_C))
    print('The False Positive Rate for African American and Caucasian are %3f and %3f, and D_FPR=%3f.'%(FPR_AA,FPR_C,FPR_AA-FPR_C))
    print('The False Negative Rate for African American and Caucasian are %3f and %3f, and D_FNR=%3f.'%(FNR_AA,FNR_C,FNR_AA-FNR_C))
    print('\n',"#"*80)
#print(y_pred.shape)
evaluation_DM(X_test1,y_test,y_pred)


 ################################################################################
The accuracy of baseline model LR is: 0.924084.
The False Positive Rate for overall population is: 0.084956.
The False Negative Rate for overall population is: 0.067126.
Specifically:
Parity Check: The rate of positive estimate for African American and Caucasian are 0.423729 and 0.538462, and D_par=-0.114733.
Calibration Check: The rate of correct estimate for African American and Caucasian are 0.932203 and 0.921978, and D_cal=0.010225.
The False Positive Rate for African American and Caucasian are 0.077465 and 0.087470, and D_FPR=-0.010006.
The False Negative Rate for African American and Caucasian are 0.053191 and 0.069815, and D_FNR=-0.016624.

 ################################################################################


If we do not put constaints on the loss function, the accuracy of the logistic regression model is around $93\%$, while the FPR, FNR is around 0.05 to 0.1. 

Then, we put the constraint in the following model:

In [ ]:
# Constaints on loss function
np.random.seed(5243)
theta1 = cvx.Variable(X_train1.shape[1])
theta1.value = np.random.rand(theta.shape[0])

tau, mu, EPS = 0.5, 1.6, 1e-4 

def g_theta(y,X,theta):
    y = 2*y - 1
    d = matmul(X,theta)
    y_d = multiply(y,d)
    return minimum(np.zeros_like(y_d),y_d)

c = 0.05
N0 = X_train1_AA.shape[0]
N1 = X_train1_C.shape[0]
N = X_train1.shape[0]
print(N,N0,N1)


Prob2 = cvx.Problem(Minimize(lossfunc(X_train1,theta1,y_train)),
                 [N0/N*sum(g_theta(y_train_C,X_train1_C,theta1)) <= c + N1/N*sum(g_theta(y_train_AA, X_train1_AA,theta1)), 
                  N0/N*sum(g_theta(y_train_C,X_train1_C,theta1)) >= N1/N*sum(g_theta(y_train_AA, X_train1_AA,theta1)) - c]) # With constraints
print(dccp.is_dccp(Prob2))
result1 = Prob2.solve(method='dccp', tau=tau, mu=mu, tau_max=1e10, verbose=True)
#g_theta(y_train,X_train1, theta.value).value.shape
#constraint()
#X_train.
#pd.DataFrame(X_train)
#pd.DataFrame(x_train1)

4584 2745 1839
True


In [ ]:
y_pred1 = predict(X_test1, theta1.value)
evaluation_DM(X_test1,y_test,y_pred1)


 ################################################################################
The accuracy of baseline model LR is: 0.924084.
The False Positive Rate for overall population is: 0.088496.
The False Negative Rate for overall population is: 0.063683.
Specifically:
Parity Check: The rate of positive estimate for African American and Caucasian are 0.427966 and 0.541758, and D_par=-0.113792.
Calibration Check: The rate of correct estimate for African American and Caucasian are 0.927966 and 0.923077, and D_cal=0.004889.
The False Positive Rate for African American and Caucasian are 0.084507 and 0.089835, and D_FPR=-0.005327.
The False Negative Rate for African American and Caucasian are 0.053191 and 0.065708, and D_FNR=-0.012517.

 ################################################################################


From above result, we may see the $DM_{sen}$ algorithm slightly drops the $D_{FNR}$ to around -0.015, which is very close to $D_{FPR}$. 

Then we implement $DM$:

In [ ]:
X_train1_sen = np.delete(X_train1,2,1)
X_train1_AA_sen = X_train1_sen[train_idx_AA]
X_train1_C_sen = X_train1_sen[train_idx_C]
X_test1_sen = np.delete(X_test1,2,1)
X_test1_AA_sen = X_test1_sen[test_idx_AA]
X_test1_C_sen = X_test1_sen[test_idx_C]

print('\n',"#"*80,'\n',' '*20," Split up Train-Test sets ",'\n',"#"*80,'\n')
print(" X_train1_sen size: ", X_train1_sen.shape, ",      y_train size: ", y_train.shape, '\n',
      "X_test1_sen size: ", X_test1_sen.shape, ',       y_test size: ',y_test.shape)
print(" X_train1_AA_sen size: ", X_train1_AA_sen.shape, ",   X_train1_C_sen size: ", X_train1_C_sen.shape, '\n',
      "X_test1_AA_sen size: ", X_test1_AA_sen.shape, ',     X_test1_C_sen size: ',X_test1_C_sen.shape, '\n',
      "Ratio:", X_train1_AA_sen.shape[0]/X_train1_C_sen.shape[0],X_test1_AA_sen.shape[0]/X_test1_C_sen.shape[0])
print('\n',"#"*80)


 ################################################################################ 
                       Split up Train-Test sets  
 ################################################################################ 

 X_train1_sen size:  (4584, 15) ,      y_train size:  (4584,) 
 X_test1_sen size:  (1146, 15) ,       y_test size:  (1146,)
 X_train1_AA_sen size:  (2745, 15) ,   X_train1_C_sen size:  (1839, 15) 
 X_test1_AA_sen size:  (688, 15) ,     X_test1_C_sen size:  (458, 15) 
 Ratio: 1.4926590538336053 1.502183406113537

 ################################################################################


In [ ]:
# Constaints on loss function
np.random.seed(5243)
theta2 = cvx.Variable(X_train1_sen.shape[1])
theta2.value = np.random.rand(theta2.shape[0])

tau, mu, EPS = 0.5, 1.6, 1e-4 
c = 0.05
N0 = X_train1_AA_sen.shape[0]
N1 = X_train1_C_sen.shape[0]
N = X_train1_sen.shape[0]
print(N,N0,N1)


Prob2 = cvx.Problem(Minimize(lossfunc(X_train1_sen,theta2,y_train)),
                 [N0/N*sum(g_theta(y_train_C,X_train1_C_sen,theta2)) <= c + N1/N*sum(g_theta(y_train_AA,X_train1_AA_sen,theta2)), 
                  N0/N*sum(g_theta(y_train_C,X_train1_C_sen,theta2)) >= N1/N*sum(g_theta(y_train_AA,X_train1_AA_sen,theta2)) - c]) # With constraints
print(dccp.is_dccp(Prob2))
result1 = Prob2.solve(method='dccp', tau=tau, mu=mu, tau_max=1e10, verbose=True)

4584 2745 1839
True


In [ ]:
y_pred2 = predict(X_test1_sen, theta2.value)
evaluation_DM(X_test1_sen,y_test,y_pred2)


 ################################################################################
The accuracy of baseline model LR is: 0.925829.
The False Positive Rate for overall population is: 0.084956.
The False Negative Rate for overall population is: 0.063683.
Specifically:
Parity Check: The rate of positive estimate for African American and Caucasian are 0.423729 and 0.540659, and D_par=-0.116931.
Calibration Check: The rate of correct estimate for African American and Caucasian are 0.932203 and 0.924176, and D_cal=0.008028.
The False Positive Rate for African American and Caucasian are 0.077465 and 0.087470, and D_FPR=-0.010006.
The False Negative Rate for African American and Caucasian are 0.053191 and 0.065708, and D_FNR=-0.012517.

 ################################################################################


In [ ]:
end = time.time()
print('The running time of overall algorithm is: %3fs.'%(end-start))

The running time of overall algorithm is: 642.106358s.


We may see $DM$ algorithm drops $D_{FNR}$. but from the above results, it's hard for us to see which one is perfect and how $DM_{sen}$ violates the disparate treatment. Overall speaking, this algorithm has an impact on controlling the difference in FPR and FNR, but the effect deserves further study as when few features are in the model, both two algorithms seem to have no effect on controlling our target.

# 3. A6

In [68]:
# X_ALL = all predictors, S = sensitive attributes, E = explanatory attribute, Y = response
S = "race"
E = "c_charge_degree"
Y = "two_year_recid"

X_ALL = ['age', 'juv_fel_count', 'decile_score', 'juv_misd_count', 
         'juv_other_count', 'priors_count', 'c_days_from_compas', 'los', 'custody', 'lasts',
         'sex', 'race', 'c_charge_degree', 'is_violent_recid', 'event']

X_train = train[X_ALL]
y_train = train[Y]

X_test = test[X_ALL]
y_test = test[Y]

## 3.1 Baseline Model

We use a logistic regression classifier.

In [70]:
clf = LogisticRegression(random_state=0).fit(X_train, y_train)
clf.score(X_test, y_test)

0.9240837696335078

## 3.2 A6 Algorithms

A6 uses the following setup:

- There is a sensitive attribute S. We want to avoid discrimination between different values of S. In our case, S is race, which takes values in {African American (1), Caucasian (0)}
- There is an explanatory variable E, which is correlated with S. Hence, simply removing S won't fix bias. In our case, we assume it is c_charge_degree.

### 3.2.1 Create functions used in pseudocode

In [71]:
# Creates a list of partitions: 1 for each unique value of e

# X is the full dataset (in our case, train)
# e is the 
def PARTITION(X):
    partitions = list()
    
    for e_i in np.unique(X[E]):
        partitions.append(X[X[E]==e_i])
    
    return partitions

In [72]:
# Delta function returns the number of observations (i.e. people) who are incorrectly classified 
# based on theoretical probabilities of reciding, calculated as the average rate of reciding
# for each explanatory varaible (in our case, type of crime comittied, c_charge_degree)

def DELTA(X, X_ei, s_i):
    
    # Gi is the number of observations for each race
    # Don't we need to pass S as a parameter for this function?
    Gi = sum(X_ei[S] == s_i)
    
    # X_ei_si is the dataset that contains the observations for each race
    X_ei_si = X_ei[X_ei[S] == s_i]
    
    # P_denom is the number of people in group 
    # P_num is number of observations who recid
    P_denom = X_ei_si.shape[0]
    P_num = sum(X_ei_si[Y] == 1)
    
    # P is the probability of reciding for one race
    # It is calculated by taking number of people who recid in each group 
    # dividied by total number of people in that group
    P = P_num/P_denom
    
    # All other observations (for the other group)
    X_ei_not_si = X_ei[X_ei[S] != s_i]
    
    # The probability of reciding for the other group (same calculation as above)
    Ps_2 = sum(X_ei_not_si[Y] == 1)/X_ei_not_si.shape[0]
    
    # Ps is P*, which is the theoretical true probability of reciding
    # Calculated by the average 
    Ps = (P+Ps_2)/2
    
    # Calcualte the number of incorrectly classified people
    d = int(round(Gi * abs(P - Ps)))
    
    return(d)

### 3.2.2 Local Massaging

In [73]:
relabeled_X_ei = list()

for X_ei in PARTITION(train):
    X_ei_copy = X_ei.copy()
    
    ranker_model = LogisticRegression(random_state=0).fit(X_ei[X_ALL], X_ei[Y])
    
    afam_index = [i for (i, v) in zip(list(range(X_ei.shape[0])), list(X_ei[S] == 1)) if v]
    afam = X_ei[X_ei[S] == 1].copy()
    delta_afam = DELTA(train, X_ei, 1)
    afam_predicted_1_index = [afam_index[v] for v in np.squeeze(np.where(ranker_model.predict(afam[X_ALL]) == 1))]
    afam_predicted_1_index_Y1 = [i for (i,v) in zip(afam_predicted_1_index, X_ei.iloc[afam_predicted_1_index][Y]) if v==1]
    afam_predicted_1 = X_ei.iloc[afam_predicted_1_index_Y1]
    
    afam_ranks = (ss.rankdata(ranker_model.decision_function(afam_predicted_1[X_ALL]))-1).astype(int)
    afam_tochange = [i for (i, v) in zip(list(range(len(afam_ranks))), afam_ranks < delta_afam) if v]
    afam_tochange_idx = [afam_predicted_1_index_Y1[v] for v in afam_tochange]
    
    cauca_index = [i for (i, v) in zip(list(range(X_ei.shape[0])), list(X_ei[S] == 0)) if v]
    cauca = X_ei[X_ei[S] == 0].copy()
    delta_cauca = DELTA(train, X_ei, 0)
    cauca_predicted_0_index = [cauca_index[v] for v in np.squeeze(np.where(ranker_model.predict(cauca[X_ALL]) == 0))]
    cauca_predicted_0_index_Y0 = [i for (i,v) in zip(cauca_predicted_0_index, X_ei.iloc[cauca_predicted_0_index][Y]) if v==0]
    cauca_predicted_0 = X_ei.iloc[cauca_predicted_0_index_Y0]
    
    cauca_ranks = (ss.rankdata(-ranker_model.decision_function(cauca_predicted_0[X_ALL]))-1).astype(int)
    cauca_tochange = [i for (i, v) in zip(list(range(len(cauca_ranks))), cauca_ranks < delta_cauca) if v]
    cauca_tochange_idx = [cauca_predicted_0_index_Y0[v] for v in cauca_tochange]
    
    for i in afam_tochange_idx:
        X_ei_copy.loc[X_ei_copy.index[i], Y] = 0
    for i in cauca_tochange_idx:
        X_ei_copy.loc[X_ei_copy.index[i], Y] = 1
    
    relabeled_X_ei.append(X_ei_copy)
    
    print("DELTA(African American) = ", delta_afam, "African Americans changed from 1 to 0")
    print("DELTA(Caucasian) = ", delta_cauca, "Caucasians changed from 0 to 1")
    
local_massaging = pd.concat(relabeled_X_ei)

DELTA(African American) =  48 African Americans changed from 1 to 0
DELTA(Caucasian) =  43 Caucasians changed from 0 to 1
DELTA(African American) =  94 African Americans changed from 1 to 0
DELTA(Caucasian) =  54 Caucasians changed from 0 to 1


In [74]:
lm_X_train = local_massaging[X_ALL]
lm_Y_train = local_massaging[Y]

In [75]:
clf = LogisticRegression(random_state=0).fit(lm_X_train, lm_Y_train)
clf.score(X_test[X_ALL], y_test)

0.918848167539267

In [76]:
# Total number changed values should be sum of all DELTAs shown above
res = [1 for i, j in zip(train.sort_index()["two_year_recid"], pd.DataFrame(lm_Y_train).sort_index()["two_year_recid"]) if i != j]
sum(res)

239

### 3.2.3 Local Preferential Sampling

In this algorithm, we take in a dataset (train) and return a modified dataset of same size.

In [77]:
pd.options.mode.chained_assignment = None
recomp_train = pd.DataFrame()

# for each partition (explanatory variable)
for X_ei in PARTITION(train):
    
    print("start partition")
    X_ei_copy = X_ei.copy()
    print("X_ei shape:", X_ei_copy.shape)
    
    # learn a ranker Hi : Xi -> Yi
    ranker_model = LogisticRegression(random_state=0).fit(X_ei[X_ALL], X_ei[Y])
    
    # Calculate half delta (AA: S_i = 1, AA: S_i = 0)
    half_delta_afam = DELTA(train, X_ei, 1) // 2
    half_delta_cauc = DELTA(train, X_ei, 0) // 2
    print("Half Delta(AA):", half_delta_afam)
    print("Half Delta(Cauc):", half_delta_cauc)
    
    # store indicies
    afam_index = [i for (i, v) in zip(list(range(X_ei.shape[0])), list(X_ei[S] == 1)) if v]
    c_index = [i for (i, v) in zip(list(range(X_ei.shape[0])), list(X_ei[S] == 0)) if v]
    print("Total AAs:", len(afam_index))
    print("Total Cs:", len(c_index))
    
    # get subset of data to work with
    afam = X_ei[X_ei[S] == 1].copy()
    c = X_ei[X_ei[S] == 0].copy()
    print("afam dataset shape:", afam.shape)
    print("c dataset shape:", c.shape)
    
    # rank AA
    afam.reset_index(drop=True, inplace=True)
    rank = pd.DataFrame(ranker_model.decision_function(afam[X_ALL]), columns = ['rank'])
    afam_with_rank = pd.concat([afam, rank], axis=1)
    
    # rank C
    c.reset_index(drop=True, inplace=True)
    rank = pd.DataFrame(ranker_model.decision_function(c[X_ALL]), columns = ['rank'])
    c_with_rank = pd.concat([c, rank], axis=1)
    
    # sort values, reset indices
    afam_with_rank = afam_with_rank.sort_values(['rank'])
    afam_with_rank.reset_index(drop = True, inplace = True)
    c_with_rank = c_with_rank.sort_values(['rank'])
    c_with_rank.reset_index(drop = True, inplace = True)
    
    ######## Modify AA data - find rows to delete/duplicate; decision boundary is 0 #####
    recid = sum(afam_with_rank['rank'] > 0)
    no_recid = sum(afam_with_rank['rank'] < 0)    
    total = len(afam_with_rank)
    
    # make copy of recids and no_recids
    # compas = compas[compas['days_b_screening_arrest'] >= -30]
    cleaned_recid = afam_with_rank[afam_with_rank['rank'] > 0]
    cleaned_no_recid = afam_with_rank[afam_with_rank['rank'] < 0]
    
    # delete first 1/2 delta values from recid
    N = half_delta_afam
    print("N:", N)
    print("rows in cleaned_recid before:", cleaned_recid.shape)
    cleaned_recid.drop(index=cleaned_recid.index[:N], axis=0, inplace=True)
    print("rows in cleaned_recid after:", cleaned_recid.shape)
    
    # flip order, then duplicate first 1/2 delta values from no_recid
    #print("cleaned_no_recid before:", cleaned_no_recid)
    cleaned_no_recid = cleaned_no_recid.sort_values(by='rank', ascending=False)
    #print("cleaned_no_recid after:", cleaned_no_recid)
    print("N:", N)
    print("rows in cleaned_no_recid before:", cleaned_no_recid.shape)
    cleaned_no_recid = cleaned_no_recid.append(cleaned_no_recid[0:N])
    print("rows in cleaned_no_recid after:", cleaned_no_recid.shape)
    
    # combine 
    total_AA = pd.concat([cleaned_recid, cleaned_no_recid])
    print("size of final A:", total_AA.shape)
    
    ########## Modify C data ############
    # Find rows to delete/duplicate; decision boundary is 0; opposite code as above
    recid = sum(c_with_rank['rank'] < 0)
    no_recid = sum(c_with_rank['rank'] > 0)    
    total = len(c_with_rank)
    
    # make copy of recids and no_recids
    cleaned_recid = c_with_rank[c_with_rank['rank'] < 0]
    cleaned_no_recid = c_with_rank[c_with_rank['rank'] > 0]
    
    # delete first 1/2 delta values from recid
    M = half_delta_cauc
    print("M:", M)
    print("rows in cleaned_recid before:", cleaned_recid.shape)
    cleaned_recid.drop(index=cleaned_recid.index[:M], axis=0, inplace=True)
    print("rows in cleaned_recid after:", cleaned_recid.shape)
    
    # flip order, then duplicate first 1/2 delta values from no_recid
    #print("cleaned_no_recid before:", cleaned_no_recid)
    cleaned_no_recid = cleaned_no_recid.sort_values(by='rank', ascending=False)
    #print("cleaned_no_recid after:", cleaned_no_recid)
    print("M:", M)
    print("rows in cleaned_no_recid before:", cleaned_no_recid.shape)
    cleaned_no_recid = cleaned_no_recid.append(cleaned_no_recid[0:M])
    print("rows in cleaned_no_recid after:", cleaned_no_recid.shape)
    
    # combine 
    total_C = pd.concat([cleaned_recid, cleaned_no_recid])
    print("size of final C:", total_C.shape)
    print("end partition")
    
    # combine both datasets
    recomp_train = recomp_train.append(total_AA)
    recomp_train = recomp_train.append(total_C)
    recomp_train = recomp_train.drop('rank', axis=1)
    
    print("size of train:", train.shape)
    print("size of recomp:", recomp_train.shape)
    

start partition
X_ei shape: (1540, 16)
Half Delta(AA): 24
Half Delta(Cauc): 21
Total AAs: 812
Total Cs: 728
afam dataset shape: (812, 16)
c dataset shape: (728, 16)
N: 24
rows in cleaned_recid before: (375, 17)
rows in cleaned_recid after: (351, 17)
N: 24
rows in cleaned_no_recid before: (437, 17)
rows in cleaned_no_recid after: (461, 17)
size of final A: (812, 17)
M: 21
rows in cleaned_recid before: (476, 17)
rows in cleaned_recid after: (455, 17)
M: 21
rows in cleaned_no_recid before: (252, 17)
rows in cleaned_no_recid after: (273, 17)
size of final C: (728, 17)
end partition
size of train: (4584, 16)
size of recomp: (1540, 16)
start partition
X_ei shape: (3044, 16)
Half Delta(AA): 47
Half Delta(Cauc): 27
Total AAs: 1933
Total Cs: 1111
afam dataset shape: (1933, 16)
c dataset shape: (1111, 16)
N: 47
rows in cleaned_recid before: (1073, 17)
rows in cleaned_recid after: (1026, 17)
N: 47
rows in cleaned_no_recid before: (860, 17)
rows in cleaned_no_recid after: (907, 17)
size of final A

# 3.3 Evaluation

Notation: P_c stands for probability based on the classifier's predictions.

## Metrics Used:

### Parity or D_all

Parity is defined as the difference is positive prediction rates in the two race groups. Paper 6 also calls this D_all, which stands for all discrimination. Fairness calls for Parity being close to 0.

<br>
<center>Parity = |P_c(recid = 1 | race = African American) - P_c(recid = 1 | race = Caucasian)</center>
    
### Calibration

Calibration is defined as the difference in accuracies between the two race groups. Fairness calls for Calibration being close to 0.

<br>

<center>Calibration = |P_c(recid predicted correctly | race = African American) - P_c(recid predicted correctly | race = Caucasian)</center>

### Equality of Odds

Equality of odds is achieved when the difference in positive prediction rates is equal for the two race groups. Fairness calls for the following value to be close to 0 for both y in {0,1}.

<br>

<center>D_Odds = P_c(recid.hat = 1 | race = African American, recid = y) - P_c(recid.hat = 1 | race = Caucasian, recid = y)</center>



In [78]:
# X must include the sensitive feature
def PARITY(X, Y_PRED):
    s = X[S]
    
    afam = X[X[S] == 1]
    num_afam = sum(Y_PRED[X[S] == 1])
    den_afam = afam.shape[0]
    
    cauca = X[X[S] == 0]
    num_cauca = sum(Y_PRED[X[S] == 0])
    den_cauca = cauca.shape[0]
    
    print("P_c(recid = 1 | race = African American) =", num_afam/den_afam)
    print("P_c(recid = 1 | race = Caucasian) =", num_cauca/den_cauca)
    parity = abs(num_afam/den_afam - num_cauca/den_cauca)
    print("Parity =", parity)
    
    return(parity)

In [79]:
# X must include S
def CALIBRATION(X, Y_TRUE, Y_PRED):
    
    afam = X[X[S] == 1]
    Y_TRUE_afam = Y_TRUE[X[S] == 1]
    num_afam = sum([1 for (i, v) in zip(Y_TRUE_afam, Y_PRED[X[S]==1]) if i == v])
    den_afam = afam.shape[0]
    
    cauca = X[X[S] == 0]
    Y_TRUE_cauca = Y_TRUE[X[S] == 0]
    num_cauca = sum([1 for (i, v) in zip(Y_TRUE_cauca, Y_PRED[X[S]==0]) if i == v])
    den_cauca = cauca.shape[0]
    
    print("P_c(recid predicted correctly | race = African American) =", num_afam/den_afam)
    print("P_c(recid predicted correctly | race = Caucasian) =", num_cauca/den_cauca)
    calibration = abs(num_afam/den_afam - num_cauca/den_cauca)
    print("Calibration =", calibration)

In [80]:
def EQUALITY_OF_ODDS(X, Y_TRUE, Y_PRED):
    
    # S = afam, Y = 0
    X_afam_0 = X[np.logical_and(X[S]==1, Y_TRUE == 0)]
    Y_PRED_afam_0 = Y_PRED[np.logical_and(X[S]==1, Y_TRUE == 0)]
    num_afam_0 = sum([1 for i in Y_PRED_afam_0 if i == 1])
    denom_afam_0 = X_afam_0.shape[0]
    P_afam_0 = num_afam_0/denom_afam_0
    
    # S = afam, Y = 1
    X_afam_1 = X[np.logical_and(X[S]==1, Y_TRUE == 1)]
    Y_PRED_afam_1 = Y_PRED[np.logical_and(X[S]==1, Y_TRUE == 1)]
    num_afam_1 = sum([1 for i in Y_PRED_afam_1 if i == 1])
    denom_afam_1 = X_afam_1.shape[0]
    P_afam_1 = num_afam_1/denom_afam_1
    
    # S = cauca, Y = 0
    X_cauca_0 = X[np.logical_and(X[S]==0, Y_TRUE == 0)]
    Y_PRED_cauca_0 = Y_PRED[np.logical_and(X[S]==0, Y_TRUE == 0)]
    num_cauca_0 = sum([1 for i in Y_PRED_cauca_0 if i == 1])
    denom_cauca_0 = X_cauca_0.shape[0]
    P_cauca_0 = num_cauca_0/denom_cauca_0
    
    # S = cauca, Y = 1
    X_cauca_1 = X[np.logical_and(X[S]==0, Y_TRUE == 1)]
    Y_PRED_cauca_1 = Y_PRED[np.logical_and(X[S]==0, Y_TRUE == 1)]
    num_cauca_1 = sum([1 for i in Y_PRED_cauca_1 if i == 1])
    denom_cauca_1 = X_cauca_1.shape[0]
    P_cauca_1 = num_cauca_1/denom_cauca_1
    
    print("For recid = 0:\n")
    print("P_c(recid.hat = 1 | race = African American, recid = 0) = ", P_afam_0)
    print("P_c(recid.hat = 1 | race = Caucasian, recid = 0) = ", P_cauca_0)
    print("Difference in odds of true recid = 0 is  = D_FPR =", abs(P_afam_0 - P_cauca_0))
    print("\n\n")
    print("For recid = 1:\n")
    print("P_c(recid.hat = 1 | race = African American, recid = 1) = ", P_afam_1)
    print("P_c(recid.hat = 1 | race = Caucasian, recid = 1) = ", P_cauca_1)
    print("Difference in odds of true recid = 1 is = D_TPR =", abs(P_afam_1 - P_cauca_1))
    


In [81]:
def D_FNR(X, Y_TRUE, Y_PRED):
    # S = afam, Y = 1
    X_afam_1 = X[np.logical_and(X[S]==1, Y_TRUE == 1)]
    Y_PRED_afam_1 = Y_PRED[np.logical_and(X[S]==1, Y_TRUE == 1)]
    num_afam_1 = sum([1 for i in Y_PRED_afam_1 if i == 0])
    denom_afam_1 = X_afam_1.shape[0]
    P_afam_1 = num_afam_1/denom_afam_1
    
    # S = cauca, Y = 1
    X_cauca_1 = X[np.logical_and(X[S]==0, Y_TRUE == 1)]
    Y_PRED_cauca_1 = Y_PRED[np.logical_and(X[S]==0, Y_TRUE == 1)]
    num_cauca_1 = sum([1 for i in Y_PRED_cauca_1 if i == 0])
    denom_cauca_1 = X_cauca_1.shape[0]
    P_cauca_1 = num_cauca_1/denom_cauca_1
    
    print("Difference in False Negative Rates")

    print("For recid = 1:\n")
    print("P_c(recid.hat = 0 | race = African American, recid = 1) = ", P_afam_1)
    print("P_c(recid.hat = 0 | race = Caucasian, recid = 1) = ", P_cauca_1)
    
    print("D_FNR =", abs(P_afam_1 - P_cauca_1))

## 3.3.1 Baseline Evaluation

In [82]:
clf = LogisticRegression(random_state=0).fit(X_train, y_train)
baseline_pred = clf.predict(X_test[X_ALL])
clf.score(X_test, y_test)

0.9240837696335078

In [83]:
print(classification_report(y_test, clf.predict(X_test[X_ALL])))

              precision    recall  f1-score   support

           0       0.93      0.92      0.92       565
           1       0.92      0.93      0.93       581

    accuracy                           0.92      1146
   macro avg       0.92      0.92      0.92      1146
weighted avg       0.92      0.92      0.92      1146



In [84]:
# Parity

PARITY(X_test, baseline_pred)

P_c(recid = 1 | race = African American) = 0.5857558139534884
P_c(recid = 1 | race = Caucasian) = 0.40829694323144106
Parity = 0.17745887072204736


0.17745887072204736

In [85]:
# Calibration

CALIBRATION(X_test, y_test, baseline_pred)

P_c(recid predicted correctly | race = African American) = 0.9142441860465116
P_c(recid predicted correctly | race = Caucasian) = 0.9388646288209607
Calibration = 0.02462044277444908


In [86]:
# Equality of Odds

EQUALITY_OF_ODDS(X_test, y_test, baseline_pred)

For recid = 0:

P_c(recid.hat = 1 | race = African American, recid = 0) =  0.11301369863013698
P_c(recid.hat = 1 | race = Caucasian, recid = 0) =  0.054945054945054944
Difference in odds of true recid = 0 is  = D_FPR = 0.058068643685082036



For recid = 1:

P_c(recid.hat = 1 | race = African American, recid = 1) =  0.9343434343434344
P_c(recid.hat = 1 | race = Caucasian, recid = 1) =  0.9297297297297298
Difference in odds of true recid = 1 is = D_TPR = 0.004613704613704583


In [87]:
# D_FNR

D_FNR(X_test, y_test, baseline_pred)

Difference in False Negative Rates
For recid = 1:

P_c(recid.hat = 0 | race = African American, recid = 1) =  0.06565656565656566
P_c(recid.hat = 0 | race = Caucasian, recid = 1) =  0.07027027027027027
D_FNR = 0.004613704613704611


## 3.3.2 Local Massaging Evaluation

In [88]:
clf = LogisticRegression(random_state=0).fit(lm_X_train, lm_Y_train)
lm_pred = clf.predict(X_test[X_ALL])
clf.score(X_test[X_ALL], y_test)

0.918848167539267

In [89]:
print(classification_report(y_test, clf.predict(X_test[X_ALL])))

              precision    recall  f1-score   support

           0       0.91      0.92      0.92       565
           1       0.93      0.91      0.92       581

    accuracy                           0.92      1146
   macro avg       0.92      0.92      0.92      1146
weighted avg       0.92      0.92      0.92      1146



In [90]:
# Parity

PARITY(X_test, lm_pred)

P_c(recid = 1 | race = African American) = 0.5494186046511628
P_c(recid = 1 | race = Caucasian) = 0.4279475982532751
Parity = 0.12147100639788766


0.12147100639788766

In [91]:
# Calibration

CALIBRATION(X_test, y_test, lm_pred)

P_c(recid predicted correctly | race = African American) = 0.9069767441860465
P_c(recid predicted correctly | race = Caucasian) = 0.9366812227074236
Calibration = 0.029704478521377142


In [92]:
# Equality of Odds

EQUALITY_OF_ODDS(X_test, y_test, lm_pred)

For recid = 0:

P_c(recid.hat = 1 | race = African American, recid = 0) =  0.07876712328767123
P_c(recid.hat = 1 | race = Caucasian, recid = 0) =  0.07326007326007326
Difference in odds of true recid = 0 is  = D_FPR = 0.005507050027597965



For recid = 1:

P_c(recid.hat = 1 | race = African American, recid = 1) =  0.8964646464646465
P_c(recid.hat = 1 | race = Caucasian, recid = 1) =  0.9513513513513514
Difference in odds of true recid = 1 is = D_TPR = 0.054886704886704885


In [93]:
# D_FNR

D_FNR(X_test, y_test, lm_pred)

Difference in False Negative Rates
For recid = 1:

P_c(recid.hat = 0 | race = African American, recid = 1) =  0.10353535353535354
P_c(recid.hat = 0 | race = Caucasian, recid = 1) =  0.04864864864864865
D_FNR = 0.054886704886704885


## 3.3.3 Local Preferential Sampling Evaluation

In [94]:
recomp_X_train = recomp_train[X_ALL]
recomp_Y_train = recomp_train[Y]
print("size of recomp_X_train:", recomp_X_train.shape)
print("size of recomp_Y_train:", recomp_Y_train.shape)

clf_LPS = LogisticRegression(random_state=0).fit(recomp_X_train, recomp_Y_train)
LPS_pred = clf_LPS.predict(X_test[X_ALL])
clf_LPS.score(X_test[X_ALL], y_test)

size of recomp_X_train: (4584, 15)
size of recomp_Y_train: (4584,)


0.9232111692844677

In [95]:
print(classification_report(y_test, clf_LPS.predict(X_test[X_ALL])))

              precision    recall  f1-score   support

           0       0.93      0.92      0.92       565
           1       0.92      0.93      0.92       581

    accuracy                           0.92      1146
   macro avg       0.92      0.92      0.92      1146
weighted avg       0.92      0.92      0.92      1146



In [96]:
PARITY(X_test, LPS_pred)

P_c(recid = 1 | race = African American) = 0.5857558139534884
P_c(recid = 1 | race = Caucasian) = 0.40611353711790393
Parity = 0.17964227683558448


0.17964227683558448

In [97]:
CALIBRATION(X_test, y_test, LPS_pred)

P_c(recid predicted correctly | race = African American) = 0.9142441860465116
P_c(recid predicted correctly | race = Caucasian) = 0.9366812227074236
Calibration = 0.022437036660912013


In [98]:
EQUALITY_OF_ODDS(X_test, y_test, LPS_pred)

For recid = 0:

P_c(recid.hat = 1 | race = African American, recid = 0) =  0.11301369863013698
P_c(recid.hat = 1 | race = Caucasian, recid = 0) =  0.054945054945054944
Difference in odds of true recid = 0 is  = D_FPR = 0.058068643685082036



For recid = 1:

P_c(recid.hat = 1 | race = African American, recid = 1) =  0.9343434343434344
P_c(recid.hat = 1 | race = Caucasian, recid = 1) =  0.9243243243243243
Difference in odds of true recid = 1 is = D_TPR = 0.010019110019110045


In [99]:
# D_FNR

D_FNR(X_test, y_test, LPS_pred)

Difference in False Negative Rates
For recid = 1:

P_c(recid.hat = 0 | race = African American, recid = 1) =  0.06565656565656566
P_c(recid.hat = 0 | race = Caucasian, recid = 1) =  0.07567567567567568
D_FNR = 0.010019110019110017
